In [ ]:
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
#데이터 불러오기
move1 = pd.read_csv('./dataset/move_as-pcs1kr.csv')
move2 = pd.read_csv('./dataset/move_as-pcs2kr.csv')
move3 = pd.read_csv('./dataset/move_as-pcs3kr.csv')
move4 = pd.read_csv('./dataset/move_as-kr-bsc20.csv')

#맵 분리
move1_erangel = move1[move1['map_name'] == 'Baltic_Main']
move2_erangel = move2[move2['map_name'] == 'Baltic_Main']
move3_erangel = move3[move3['map_name'] == 'Baltic_Main']
move4_erangel = move4[move4['map_name'] == 'Baltic_Main']
move1_miramar = move1[move1['map_name'] == 'Desert_Main']
move2_miramar = move2[move2['map_name'] == 'Desert_Main']
move3_miramar = move3[move3['map_name'] == 'Desert_Main']
move4_miramar = move4[move4['map_name'] == 'Desert_Main']

#합치기
move_erangel = pd.concat([move1_erangel,move2_erangel,move3_erangel,move4_erangel], ignore_index=True)
move_miramar = pd.concat([move1_miramar,move2_miramar,move3_miramar,move4_miramar], ignore_index=True)

outlier = [
    'f566ed1a-9187-4937-a435-3b31ed881f2d',
    '3bb55936-e760-45eb-a18d-3e8f50908f8b',
    '618291a3-d2a6-4b33-be8a-0bcd96e7760a'
]

#이상치 제거
move_erangel = move_erangel[~move_erangel['match_id'].isin(outlier)]
move_miramar = move_miramar[~move_miramar['match_id'].isin(outlier)]

#map_name 제거
move_erangel = move_erangel.drop(columns='map_name')
move_miramar = move_miramar.drop(columns='map_name')

#pro 데이터 합치기
pro0 = pd.read_csv('./dataset/pro_ksc.csv')
pro1 = pd.read_csv('./dataset/pro_data_as-pcs1kr.csv')
pro2 = pd.read_csv('./dataset/pro_data_as-pcs2kr.csv')
pro3 = pd.read_csv('./dataset/pro_data_as-pcs3kr.csv')
pro = pd.concat([pro0, pro1, pro2, pro3], ignore_index=True)
pro_miramar = pro[pro['map_name']=='Miramar']

pro_miramar.to_csv('./dataset/pro_data_miramar.csv', index=False)

move_miramar['count'] = 0
#맵 크기 : 816000 * 816000
map_size = 816000

def binning_move(move_data, bin_num):  # bin_num단위로 나눔

    move_data['y'] = map_size - move_data['y']  # y좌표 반전
    move_data['x'] = (move_data['x'] / bin_num).round().astype(int)
    move_data['y'] = (move_data['y'] / bin_num).round().astype(int)
    move_data['z'] = move_data['z'].astype(int)


def timestamp(move_data):  # timestamp 초 단위로 정리
    move_data['timestamp'] = pd.to_datetime(move_data['timestamp'])
    hour = move_data['timestamp'].dt.hour * 60 * 60
    minute = move_data['timestamp'].dt.minute * 60
    second = move_data['timestamp'].dt.second
    move_data['timestamp'] = hour + minute + second

def filter_by_time(data):  # 초 단위로 나눠야함
    first_timestamp = data['timestamp'].min()
    data['timestamp'] -= first_timestamp
    return data[(data['timestamp'] >= 720) & (data['timestamp'] <= 1620)]

binning_move(move_miramar, 4000)
timestamp(move_miramar)
move_miramar = move_miramar.groupby(['match_id', 'player_name']).apply(filter_by_time).reset_index(drop=True)
move_miramar.to_csv('./dataset/move_miramar_40.csv', index=False)

In [ ]:
def count_sum(move_data, match_num, min_num):
    map_size = 204

    all_size = np.zeros((map_size+1, map_size+1))
    for index, row in move_data.iterrows():
        x = int(row['x'])
        y = int(row['y'])

        # x, y 인덱스가 배열 크기를 초과하는지 확인
        if x >= map_size or y >= map_size:
            print(index,x,y)
            continue

        all_size[x, y] += 1

    df = []
    for x in range(map_size + 1):
        for y in range(map_size + 1):
            df.append([x, y, all_size[x, y]])

    df = pd.DataFrame(df, columns=['x', 'y', 'count'])
    df['count'] /= match_num
    df = df[df['count'] >= min_num]

    return df

match_num = len(move_miramar['match_id'].unique())

count_data = count_sum(move_miramar, match_num, 2)

count_data = count_data[
    ~(
        ((count_data['x'] >= 40) & (count_data['x'] <= 55) & (count_data['y'] >= 100) & (count_data['y'] <= 110)) | # Monte Nuevo
        ((count_data['x'] >= 85) & (count_data['x'] <= 100) & (count_data['y'] >= 125) & (count_data['y'] <= 135)) | # San Martin
        ((count_data['x'] >= 87) & (count_data['x'] <= 97) & (count_data['y'] >= 92) & (count_data['y'] <= 102)) | # Pecado
        ((count_data['x'] >= 155) & (count_data['x'] <= 165) & (count_data['y'] >= 135) & (count_data['y'] <= 145)) | # El Azahar
        ((count_data['x'] >= 155) & (count_data['x'] <= 165) & (count_data['y'] >= 84) & (count_data['y'] <= 93)) | # Impala
        ((count_data['x'] >= 104) & (count_data['x'] <= 130) & (count_data['y'] >= 59) & (count_data['y'] <= 75)) | #Los Leonos
        ((count_data['x'] >= 87) & (count_data['x'] <= 97) & (count_data['y'] >= 92) & (count_data['y'] <= 102))
    )
]

count_data.to_csv('./dataset/count_data.csv', index=False)

In [ ]:
def plot_count(data, image_path):
    map_size = 204

    map_img = mpimg.imread(image_path)
    fig, ax = plt.subplots(figsize=(30, 30), dpi=100)

    ax.scatter(data['x'], data['y'], s=data['count'] * 5, c='red')

    ax.imshow(map_img, extent=[0, map_size, 0, map_size], aspect='auto')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.title('route')
    ax.set_xticks(np.arange(0, map_size, 5))
    ax.set_yticks(np.arange(0, map_size, 5))
    ax.grid(True, which='both', color='white', linestyle='-', linewidth=0.5)
    plt.show()

plot_count(count_data, 'miramar.jpg')

In [ ]:
#12분부터 25분까지 슬라이싱
move_miramar = move_miramar[(move_miramar['timestamp'] >= 720) & (move_miramar['timestamp'] <= 1560)]

for idx, row in move_miramar.iterrows():
    # 일치하는 행이 있는지 검사
    if any((count_data['x'] == row['x']) & (count_data['y'] == row['y'])):
        move_miramar.at[idx, 'count'] = 1

move_miramar['survived'] = 1

#플레이어별 team_rank뽑기
pro_miramar_rank = pro_miramar.groupby(['match_id', 'player_id'])[['team_rank']].first().reset_index()
pro_miramar_rank.rename(columns={'player_id' : 'player_name'}, inplace=True)
pro_miramar_rank.to_csv('./dataset/pro_miramar_rank.csv', index=False)

#count, survived 합계 내기
move_miramar_sum = move_miramar.groupby(['match_id', 'player_name'])[['count', 'survived']].sum().reset_index()

#합친 데이터에  team_rank 붙이기
move_miramar_sum = pd.merge(move_miramar_sum, pro_miramar_rank,  on=['match_id', 'player_name'], how='left')

#비율 추가
move_miramar_sum['ratio'] = move_miramar_sum['count'] / move_miramar_sum['survived']
move_miramar_sum.to_csv('./dataset/move_miramar_sum.csv', index=False)

In [ ]:

#평균 내기
move_miramar_mean = move_miramar_sum.groupby(['team_rank'])[['count', 'ratio']].mean().reset_index()
move_miramar_mean.to_csv('./dataset/miramar_final_mean.csv', index=False)

#시각화
plt.figure(figsize=(10, 6))
plt.scatter(move_miramar_mean['team_rank'], move_miramar_mean['ratio'], alpha=1, color='red')
plt.title('Scatter Plot of Team Rank vs Count')
plt.xlabel('Team Rank')
plt.ylabel('count')
plt.grid(True)
plt.show()


print("done")
